# Application à la classification : l’analyse d’opinions

In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from nltk import SnowballStemmer
from nltk import pos_tag
# from collections import namedtuple
# CondProb = namedtuple('CondProb', 'prior_idx likelihood_idx')

# I Implementation du classifieur

## Question 1
Compléter la fonction count_words qui va compter le nombre d’occurrences de chaque mot dans une liste de string et renvoyer le vocabulaire.

In [2]:
# %load sentimentanalysis.py
# Authors: Alexandre Gramfort
#          Chloe Clavel
# License: BSD Style.
# TP Cours ML Telecom ParisTech MDI343

import os.path as op
import numpy as np

from sklearn.base import BaseEstimator, ClassifierMixin

###############################################################################
# Load data
print("Loading dataset")

from glob import glob
filenames_neg = sorted(glob(op.join('..', 'data', 'imdb1', 'neg', '*.txt')))
filenames_pos = sorted(glob(op.join('..', 'data', 'imdb1', 'pos', '*.txt')))

texts_neg = [open(f).read() for f in filenames_neg]
texts_pos = [open(f).read() for f in filenames_pos]
texts = texts_neg + texts_pos
y = np.ones(len(texts), dtype=np.int)
y[:len(texts_neg)] = 0.

print("%d documents" % len(texts))

###############################################################################
# Start part to fill in


def count_words(texts):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts

    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
        n_samples == number of documents.
        n_features == number of words in vocabulary.
    """
    # first iteration: know number of words to initialize array
    n_samples = len(texts)
    # second version: use strip and lower to minimize vocabulary
    words = set(word.strip().lower() for text in texts
                for word in text.split())
    word_idx = dict((word, id_w) for id_w, word in enumerate(words))
    n_features = len(words)
    counts = np.zeros((n_samples, n_features))

    # second iteration: word count
    # parsing all texts
    vocabulary = dict((word, word_idx[word]) for word in words)
    for id_text, text in enumerate(texts):
        # parsing a text
        for word in text.split():
            # evaluate a word
            word_clean = word.strip().lower()
            counts[id_text][word_idx[word_clean]] = \
                counts[id_text][word_idx[word_clean]] + 1
    # counts = np.zeros((len(texts, n_features)))
    return vocabulary, counts


class NB(BaseEstimator, ClassifierMixin):
    def __init__(self, D):
        # get word dictionary
        self.D = D
        # label => array index
        self.label_idx = dict()
        # array index => label (reverse dictionary)
        self.index2label = dict()
        self.likelihood = None
        self.prior = None

    def fit(self, X, y):
        # class index dictionary:
        # class 0 -> indexes, class 1 -> indexes
        labels = set(np.unique(y))
        d_indexes = {label: np.where(y == label)[0]
                     for label in labels}
        # total samples count
        N = X.shape[0]
        # label indexing
        self.label_idx = dict((label, id_label)
                              for id_label, label in enumerate(labels))
        self.index2label = dict((self.label_idx[label], label)
                                for label in self.label_idx)
        # initialize likelihood array (n_labels x p words)
        self.likelihood = np.zeros((len(labels), X.shape[1]))
        # initialize prior array (n labels)
        self.prior = np.zeros(len(labels))
        # count all training tokens
        # countAllTokens = np.sum(X, axis=0)
        for label in d_indexes:
            # Compute prior
            self.prior[self.label_idx[label]] = d_indexes[0].shape[0] / N
            # print(self.prior)
            # Compute sum of each word for each class
            countTokensFromDoc = np.sum(X[d_indexes[label]], axis=0) + 1
            countAllTokens = np.asscalar(np.sum(countTokensFromDoc))
            # print(countTokensFromDoc)
            # Compute probability given class with Laplace smoothing
            self.likelihood[self.label_idx[label]] = \
                countTokensFromDoc / countAllTokens
            # print(self.likelihood)
        return self

    def predict(self, X):
        # compute the score of for each row of X for each class
        # #words within test file are already in matrix
        # product of word probabilities is then equivalent
        # to matrix multiplication between X[text] and log(parameters)
        prediction = X @ np.log(self.likelihood.T)
        # add the prior log
        prediction += np.log(self.prior)
        # print(prediction)

        # 1 vs all
        majority_class = np.argmax(prediction, axis=1).astype(np.int)

        # convert index -> labels
        return np.array([self.index2label[index] for index in majority_class])

        # return (np.random.randn(len(X)) > 0).astype(np.int)

    def score(self, X, y):
        return np.mean(self.predict(X) == y)


# Count words in text
vocabulary, X = count_words(texts)

# Try to fit, predict and score
nb = NB(vocabulary)
nb.fit(X[::2], y[::2])
print(nb.score(X[1::2], y[1::2]))


Loading dataset
2000 documents
0.823


In [3]:
print("Vocabulary size : {}".format(len(vocabulary)))
print("#samples : {}; #words : {}".format(X.shape[0], X.shape[1]))

Vocabulary size : 50920
#samples : 2000; #words : 50920


## Question 2
Expliquer comment les classes positives et négatives ont été assignées sur les critiques de films (voir fichier poldata.README.2.0)

Les commentaires avaient été préalablement tagués sur IMDB ainsi que sur les autres sources: lorsqu'un internaute emmet un commentaire, il doit également lui adjoindre une note entre 1 et 5 étoiles sur IMDB. 
- Une note clairement positive (4 ou 5 etoiles / 5) est considérée comme un avis positif
- Une note clairement négative (1 ou 2 étoiles / 5) est considérée comme un avis négatif

## Question 3
Compléter la classe NB pour qu’elle implémente le classifieur Naive Bayes en vous appuyant sur le pseudo-code de la figure 1 et sa documentation ci-dessous :
- le vocabulaire V correspond à l’ensemble des mots différents composant un ensemble de documents (vocabulary dans count_words)
- C correspond à l’ensemble des classes et D l’ensemble des documents
- La fonction countTokensOfTerm(text,t) représente le nombre d’occurrences d’un mot t dans un ensemble de textes text (calcul fait dans count_words)
- L’étape de lissage appelée lissage de Laplace (+1 ligne 10) permet l’attribution de probabilité non nulle à des mots qui n’interviendraient pas dans l’ensemble d’apprentissage
- La fonction ExtractTokensFromDoc(V,d) récupère le vocabulaire associé au document d.

La classe est complétée directement dans le source sentimentanalysis.py.
En pratique:
- J instancie la classe avec un dictionnaire contenant l'ensemble des mots à prendre en compte (self.D), ainsi que leur index dans le ndarray
- La fonction fit effectue l'étape de training:
 - On récupère l'ensemble des classes de y (labels)
 - On compte l'ensemble des samples (N)
 - Les mots pris en compte sont ceux du dictionnaire (D)
 - On crée une matrice contenant les probabilités d'apparition de chaque mot du dictionnaire dans une classe (len(labels) x len(D))
 - Pour chaque label, on calcule le prior (p(classe)) ainsi que la fréquence d'apparition de chaque mot dans chaque classe en effectuant un lissage de Laplace sur la probabilité
- La fonction test calcule les prédictions de labels sur l ensemble de test:
 - On se ramène à un produit matriciel pour le calcul du posterior car on passe par le logarithme du prior et des probabilités conditionnelles (on assume indépendantes)
 - La classe prédite est la classe dont le posterior est le plus grand
 

## Question 4
Evaluer les performances de votre classifieur en cross-validation 5-folds.

In [4]:
clf = NB(vocabulary)
scores = cross_val_score(clf, X, y, cv=5)
scores

array([ 0.81  ,  0.83  ,  0.8175,  0.825 ,  0.795 ])

La performance du prédicteur sur le test set est de l'ordre de 80%. Après cross validation, la meilleure valeur est à 83%

## Question 5
Modifiez la fonction count_words pour qu’elle ignore les “stop words” dans le fichier
data/english.stop. Les performances sont-elles améliorées ?

In [5]:
sw_file = "../data/english.stop"
stop_words = set()

with open(sw_file) as sw_fileReader:
    for line in sw_fileReader:
        stop_words.add(line.strip())

In [6]:
def count_words_2(texts, stop_words):
    """Vectorize text : return count of each word in the text snippets

    Parameters
    ----------
    texts : list of str
        The texts

    Returns
    -------
    vocabulary : dict
        A dictionary that points to an index in counts for each word.
    counts : ndarray, shape (n_samples, n_features)
        The counts of each word in each text.
        n_samples == number of documents.
        n_features == number of words in vocabulary.
    """
    # first iteration: know number of words to initialize array
    n_samples = len(texts)
    # second version: use strip and lower to minimize vocabulary
    # third version: remove stop words
    words = set(word.strip().lower() for text in texts
                for word in text.split()) - stop_words
    word_idx = dict((word, id_w) for id_w, word in enumerate(words))
    n_features = len(words)
    counts = np.zeros((n_samples, n_features))

    # second iteration: word count
    # parsing all texts
    vocabulary = dict((word, word_idx[word]) for word in words)
    for id_text, text in enumerate(texts):
        # parsing a text
        for word in text.split():
            # evaluate a word
            word_clean = word.strip().lower()
            if word_clean not in stop_words:
                counts[id_text][word_idx[word_clean]] = \
                    counts[id_text][word_idx[word_clean]] + 1
    return vocabulary, counts

In [7]:
# Count words in text
vocabulary_2, X_2 = count_words_2(texts, stop_words)

print("#samples : {}; #words : {}".format(X_2.shape[0], X_2.shape[1]))

# Try to fit, predict and score
clf_2 = NB(vocabulary_2)
scores = cross_val_score(clf_2, X_2, y, cv=5)
scores


#samples : 2000; #words : 50375


array([ 0.8025,  0.8125,  0.8125,  0.825 ,  0.785 ])

La performance du prédicteur ne s'est pas améliorée avec la suppression des stop words. La meilleure prévision est maintenant de 81.25%.
En rehgardant la liste des stop words, ce n'est pas forcément étonnant: on y voit des éléments qui devraient être discriminants (awfully, appropriate, sensible, useful, well) et qui sont donc retirés de l'algorithme de prédiction

# II Utilisation de scikitlearn

## Question 1
Comparer votre implémentation avec scikitlearn. On utilisera la classe CountVectorizer et un Pipeline. Vous expérimenterez en autorisant les mots et bigrammes ou en travaillant sur les sous-chaines de
caractères (option analyzer='char').

In [8]:
# Standard Pipeline, no cross validation
# We do the same first test
# Only words with size > 2 for this implementation

pipeline = Pipeline([('vectorizer', CountVectorizer()),
                     ('multinomialNB', MultinomialNB())])

pipeline.fit(texts[::2], y[::2])

test = pipeline.predict(texts[1::2])

np.mean(test == y[1::2])

0.81299999999999994

La première implémentation, sans tuner aucun paramètre, et sans cross validation est équivalente à celle de notre prédicteur custom

In [9]:
# Standard Pipeline, no cross validation
# We do the same first test
# Allow Bigrams for this implementation

pipeline = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                     ('multinomialNB', MultinomialNB())])

pipeline.fit(texts[::2], y[::2])

test = pipeline.predict(texts[1::2])

np.mean(test == y[1::2])

0.84099999999999997

La deuxième version qui autorise les bigrammes améliore le score (84%)
- On améliore la compréhension des avis en interprétant également les suites de 2 mots
- On n'est pas en situation d'overfitting car la prévision du testing set ne s'est pas effondrée

In [10]:
pipeline = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                     ('multinomialNB', MultinomialNB())])

scores = cross_val_score(pipeline, texts, y, cv=5)
print(scores)


[ 0.8175  0.8375  0.8225  0.8425  0.8325]


La prédiction sur le testing set reste stable avec la cross validation

## Question 2
Tester un autre algorithme de la librairie scikitlearn (ex : LinearSVC, LogisticRegression).

In [11]:
# Test avec LogisticRegression

pipeline = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                     ('logisticRegression', LogisticRegression())])

scores = cross_val_score(pipeline, texts, y, cv=5)
print(scores)

[ 0.8225  0.8525  0.8525  0.87    0.865 ]


En conservant les bigrammes, on obtient une prédiction légèrement meilleure avec la régression logistique (87%) 

## Question 3
Utiliser la librairie NLTK afin de procéder à une racinisation (stemming). Vous utiliserez la classe SnowballStemmer.

In [12]:
stemmer = SnowballStemmer("english")
analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc))

stem_vectorizer = CountVectorizer(analyzer=stemmed_words, ngram_range=(1, 2))

pipeline = Pipeline([('vectorizer', stem_vectorizer),
                     ('multinomialNB', MultinomialNB())])

scores = cross_val_score(pipeline, texts, y, cv=5)
print(scores)


[ 0.795   0.8125  0.8025  0.8325  0.7925]


In [13]:
print(stem_vectorizer.fit_transform(['This parrot is no more. It has ceased to be. '
                                     'It s expired and gone to meet its maker.']))
print(stem_vectorizer.get_feature_names())

  (0, 8)	1
  (0, 9)	1
  (0, 4)	1
  (0, 0)	1
  (0, 3)	1
  (0, 1)	1
  (0, 14)	2
  (0, 2)	1
  (0, 5)	1
  (0, 7)	3
  (0, 10)	1
  (0, 11)	1
  (0, 6)	1
  (0, 12)	1
  (0, 13)	1
['and', 'be', 'ceas', 'expir', 'gone', 'has', 'is', 'it', 'maker', 'meet', 'more', 'no', 'parrot', 'this', 'to']


Il n'y a pas eu de gain en terme de prédiction avec l utilisation du stemming

## Question 4
Filtrer les mots par catégorie grammaticale (POS : Part Of Speech) et ne garder que les
noms, les verbes, les adverbes et les adjectifs pour la classification.

In [ ]:
textsample = word_tokenize("They refuse to permit us to obtain the refuse permit")